In [ ]:
# Victoria Driver Licence Extractor with OCR
# This notebook detects VIC licence and card numbers,
# automatically handling text-based and image-based PDFs.


In [ ]:
# Install Dependencies!pip install --upgrade pip!pip install PyPDF2 pandas nltk pytesseract pdf2image!apt-get install -y tesseract-ocr poppler-utils

In [ ]:
# Imports and setupimport re, pandas as pdfrom PyPDF2 import PdfReaderfrom pdf2image import convert_from_pathimport pytesseract, nltk, requestsfrom nltk.corpus import stopwords as nltk_stopwordsnltk.download('stopwords')# Load stopwords (external optional list)STOPWORDS_URL = 'https://raw.githubusercontent.com/tp12121212/stopwords/main/stopwords.txt'stopwords_set = set(nltk_stopwords.words('english'))try:    r = requests.get(STOPWORDS_URL)    if r.status_code == 200:        stopwords_set.update(r.text.splitlines())except:    print('Could not fetch external stopwords, using NLTK only.')

In [ ]:
# PDF text extraction with OCR fallbackpdf_path = 'input19.pdf'  # Update this to your PDF pathfull_text = ''reader = PdfReader(pdf_path)for page_num, page in enumerate(reader.pages):    page_text = page.extract_text()    if page_text and len(page_text.strip()) > 0:        full_text += page_text + '\n'    else:        print(f"Page {page_num+1} is image-based, performing OCR...")        images = convert_from_path(pdf_path, first_page=page_num+1, last_page=page_num+1)        for img in images:            ocr_text = pytesseract.image_to_string(img)            full_text += ocr_text + '\n'print('PDF text extracted (including OCR if needed).')

In [ ]:
# Regex utility functionregex_candidates = []def find_with_lines(pattern, text, desc):    for match in re.finditer(pattern, text, flags=re.IGNORECASE):        match_text = match.group(1) if match.groups() else match.group(0)        line_number = text.count('\n', 0, match.start()) + 1        regex_candidates.append((pattern, desc, match_text, line_number))

In [ ]:
# Victoria driver licence regexes# Front licence number (VIC 8-digit format)vic_licence_regex = r"\b(?:Licence\sNo\.?|Licence\sNumber)\s*[:\-]?\s*(\d{8})\b"find_with_lines(vic_licence_regex, full_text, 'VIC_Driver_Licence_Number_Front')# Back card number (3 to 4 digits)vic_card_regex = r"\b(?:Card\sNo\.?|Card\sNumber)\s*[:\-]?\s*(\d{3,4})\b"find_with_lines(vic_card_regex, full_text, 'VIC_Driver_Licence_Card_Back')

In [ ]:
# Deduplicate and export resultsregex_dict = {}for regex, desc, sample, line in regex_candidates:    key = (regex, sample, line)    if key not in regex_dict:        regex_dict[key] = (desc, sample, line)df_regex = pd.DataFrame(    [(regex, desc, sample, line) for (regex, sample, line), (desc, sample, line) in regex_dict.items()],    columns=['RegexPattern', 'Description', 'SampleValue', 'LineNumber'])output_csv_regex = 'purview_regex.csv'df_regex.to_csv(output_csv_regex, index=False)print('Regex patterns saved to:', output_csv_regex)df_regex.head(10)